# Code to clean NREL NAATBatt's data and calculate road distances between facilities to accurately model transportation environmental impacts

__________________________________________________________________________________________________________

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import string as str
from itertools import product
import sklearn
import requests as req
import json
import time
from datetime import datetime as dt

### Import data, NAATBatt data set freely available on the NREL website. The exact version included in files here.

In [ ]:
# import naatbatt sheets for whole life cycle
df_rawm = pd.read_excel("Data/naatbatt-EDITABLE.xlsx",sheet_name='1-RawMatl')
df_battm = pd.read_excel("Data/naatbatt-EDITABLE.xlsx",sheet_name='2-Battery Grade Materials')
df_battc = pd.read_excel("Data/naatbatt-EDITABLE.xlsx",sheet_name='3-Other Battery Comps Matls')
df_cell = pd.read_excel("Data/naatbatt-EDITABLE.xlsx",sheet_name='4-Electrodes and Cells')
df_pack = pd.read_excel("Data/naatbatt-EDITABLE.xlsx",sheet_name='5-ModPack')
df_eol = pd.read_excel("Data/naatbatt-EDITABLE.xlsx",sheet_name='6-EOL')


dfs = [df_rawm,df_battm,df_battc,df_cell,df_pack,df_eol] # list of all dfs


### Clean data

In [ ]:
# lists to select rows related to LFP, for use in following lines
rawm = ['li','graphite']
battm = ['lithium','lfp','graphite']
cell = ['lfp']
eol = ['disassembling','hydro','direct']


dfs_i = []
# get only operations/facilites within the United States
for x in dfs:
    x = x[x['Facility Country'] =='US']
    dfs_i.append(x)


dfs_c = []
# get relevent columns and rows from all dfs except eol
for n in range(0,5):
    x = dfs_i[n]
    x = x[['ID', 'Status', 'Supply Chain Segment', 'Company', 'NAATBatt Member',
                'Facility Name', 'Product Type', 'Product','Facility or Company Website',
                'Facility Address', 'Facility City', 'Facility State or Province',
                'Facility Country','Latitude', 'Longitude','Production Capacity', 'Production Units']] # relevent columns
    # for each segment, only choose rows related to LFP cell
    if n == 0:
        x['Product'] = x['Product'].str.lower()
        x = x[x['Product'].str.contains('|'.join(rawm),na=False)]
    elif n == 1:
        x['Product'] = x['Product'].str.lower()
        x = x[x['Product'].str.contains('|'.join(battm),na=False)]
    elif n == 3:
        x['Product'] = x['Product'].str.lower()
        x = x[x['Product'].str.contains('|'.join(cell),na=False)]
        x = x[x['Production Units'] == 'GWh/yr'] # select rows where capacities are in GWh/yr
    elif n == 4:
        x = x[x['Production Units'] == 'GWh/yr'] # select rows where capacities are in GWh/yr

    # reset index and assign cleaned df to new list
    x.reset_index(inplace=True,drop=True)
    dfs_c.append(x)


df_eol = dfs_i[-1]
 # df_eol has different columns so do by itself, not in for loop above
df_eol = df_eol[['ID', 'Status', 'Supply Chain Segment', 'Company', 'NAATBatt Member',
                'Facility Name', 'Facility Type', 'Product', 'Facility Address',
                'Facility City', 'Facility State or Province','Facility Country',
                'Latitude', 'Longitude','Capacity', 'Capacity Units']]
# get relevent rows from eol, not in above code because of column issue
df_eol['Facility Type'] = df_eol['Facility Type'].str.lower()
df_eol = df_eol[df_eol['Facility Type'].str.contains('|'.join(eol),na=False)]


# get cleaned dfs
df_rawm = dfs_c[0]
df_battm = dfs_c[1]
df_battc = dfs_c[2]
df_cell = dfs_c[3]
df_pack = dfs_c[4]

# ignore subsetting warnings

### Save cleaned data

In [ ]:
# create a excel writer object
with pd.ExcelWriter("Data/naatbatt_CLEANED.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_rawm.to_excel(writer, sheet_name="RawMaterials", index=False)
    df_battm.to_excel(writer, sheet_name="BattMaterials", index=False)
    df_battc.to_excel(writer, sheet_name="BattComponents", index=False)
    df_cell.to_excel(writer, sheet_name="ElectrodesCells", index=False)
    df_pack.to_excel(writer, sheet_name="BattPacks", index=False)
    df_eol.to_excel(writer, sheet_name="EOL", index=False)

#### from cleaned data, facilities were researched and selected based on criteria to be accurately modeled, of significant size, and energy storage realted (e.g. wholly EV battery manufacturers weren't considered)

In [ ]:
# import facility location data
df_s = pd.read_excel("Data/naatbatt_SCENARIOS.xlsx",sheet_name='scenarios_lists') # facility names at life-cycle stages
df_ll = pd.read_excel("Data/naatbatt_SCENARIOS.xlsx",sheet_name='scenarios_Trans') # facility latitude and longitudes

value_lists = []
for column in df_s.columns[0:]:
    value_lists.append(df_s[column].to_list())

# cross multiply all the facilities to generate all possible supply routes e.g. raw material from site A is processed at refinery B is made into a battery at facility C...; does not emulate real world possibilities, overestimation of supply routes
scenarios = list(product(value_lists[0],value_lists[1],value_lists[2],value_lists[3],value_lists[4]))

df_s = pd.DataFrame(scenarios,columns=['mineLi','refineLi','mineGr','battPack','recycle'])
df_s.dropna(inplace=True)
df_s.reset_index(inplace=True,drop=True)

# use stage placeholder, use stage will be modeled as fort stockton texas only
df_s['use_placeholder'] = "use"
df_s = df_s[['mineLi','refineLi','mineGr','battPack','use_placeholder','recycle']]

lat_dict = dict(zip(df_ll['name'],df_ll['lat']))
lon_dict = dict(zip(df_ll['name'],df_ll['lon']))

# column to determine if the scenario is used or not based on real world restrictions (e.g. Hell's kitchen lithium will not be refined at albemarle's processing plant)
df_s['del'] = ' '
df_s['liSource'] = 'spodumene'

# constraints to subset scenarios to real world possibilities
for i,r in df_s.iterrows():
    if r['mineLi'] == 'Albemarle - Kings Mountain, NC' and r['refineLi'] == 'Albemarle Corporation, Silver Peak NV':
        df_s.loc[i,'del'] = 'Yes'
    elif r['mineLi'] == "Hell's Kitchen Lithium and Power Project" and r['refineLi'] != "Hell's Kitchen Lithium and Power Project":
        df_s.loc[i,'del'] = 'Yes'
    elif r['mineLi'] == 'Albemarle - Kings Mountain, NC' and r['refineLi'] == "Hell's Kitchen Lithium and Power Project":
        df_s.loc[i,'del'] = 'Yes'
    elif r['mineLi'] == 'Albemarle Corporation, Silver Peak NV' and r['refineLi'] == "Hell's Kitchen Lithium and Power Project":
        df_s.loc[i,'del'] = 'Yes'
    elif r['mineLi'] == 'Albemarle Corporation, Silver Peak NV':
        df_s.loc[i,'liSource'] = 'brine'
    elif r['mineLi'] == r['refineLi'] and r['mineLi'] == "Hell's Kitchen Lithium and Power Project":
        df_s.loc[i,'liSource'] = 'geothermal_brine'


# subset to scenarios
df_s = df_s[~(df_s['del']=='Yes')]

df_s.reset_index(inplace=True,drop=True)


        


In [ ]:
df_s

In [ ]:
################
# TRANSPORTATION DISTANCES

fort_stockton_lat = '30.8940'
fort_stockton_lon = '-102.8793'

# create columns for lat and lon coordinates
df_s['lat1'] = 0
df_s['lon1'] = 0
df_s['lat2'] = 0
df_s['lon2'] = 0
df_s['lat4'] = 0
df_s['lon4'] = 0
df_s['lat5'] = fort_stockton_lat # https://www.census.gov/geographies/reference-files/time-series/geo/centers-population.html '37.415725' '-92.346525'
df_s['lon5'] = fort_stockton_lon
df_s['lat6'] = 0
df_s['lon6'] = 0
df_s['lat3'] = 0
df_s['lon3'] = 0


# fill lat and lon coordinates to corresponding scenario locations
df_s['lat1'] = df_s['mineLi'].map(lat_dict)
df_s['lon1'] = df_s['mineLi'].map(lon_dict)
df_s['lat2'] = df_s['refineLi'].map(lat_dict)
df_s['lon2'] = df_s['refineLi'].map(lon_dict)
df_s['lat4'] = df_s['battPack'].map(lat_dict)
df_s['lon4'] = df_s['battPack'].map(lon_dict)
df_s['lat6'] = df_s['recycle'].map(lat_dict)
df_s['lon6'] = df_s['recycle'].map(lon_dict)
df_s['lat3'] = df_s['mineGr'].map(lat_dict)
df_s['lon3'] = df_s['mineGr'].map(lon_dict)


In [ ]:
# df_s

### US routes transportation distances by heavy duty road vehicles

In [ ]:
# create columns for transporation distances between segments

df_s['trans1'] = 0 # lithium source to lithium processing
df_s['trans2'] = 0 # lithium processing to battery manufacturer
df_s['trans3'] = 0 # battery manufacturer to proxy use (west Texas, Fort Stockton)
df_s['trans4'] = 0 # proxy use to battery recycling
df_s['trans5'] = 0 # graphite to battery manufacturer



## use Openrouteservice application to find traveling distances between lat and lon coordinates, make a free account, monthly use limitations

# key require for software
key = '5b3ce3597851110001cf624895482b2e40564e70a1f7588205d525d0'
#'5b3ce3597851110001cf624895482b2e40564e70a1f7588205d525d0'

# counter required for bug in openrouteservice at time of making; timeout error and need to recall API
counter = 0

for i,r in df_s.iterrows():

    # counter commands for bug
    if counter > 3:
        print(dt.now())
        time.sleep(60)
        print(dt.now())
        counter = 0

    # necessesary data for api call
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': key,
        'Content-Type': 'application/json; charset=utf-8'
    }

    # get segment distances from coordinates for all scenarios; e.g. scenario 1 raw to refining, scenario 2 refining to manufacturing, etc.
    for x in range(8,17,2):
        counter = counter + 1
        if x in [8,10,12,14]: # columns where lat and lon data are
            lat0 = r[x]
            lon0 = r[x+1]
            lat1 = r[x+2]
            lon1 = r[x+3]

            body = {"coordinates":[[lon0,lat0],[lon1,lat1]],"radiuses":[-1]}

            call = req.post('https://api.openrouteservice.org/v2/directions/driving-hgv/geojson', json=body, headers=headers)
            json_obj = call.json()
           

            trans_dist = json_obj['features'][0]['properties']['segments'][0]['distance']

            # distances in meters, get kilometers
            if x == 8:
                df_s.loc[i,'trans1'] = trans_dist/1000
            elif x == 10:
                df_s.loc[i,'trans2'] = trans_dist/1000
            elif x == 12:
                df_s.loc[i,'trans3'] = trans_dist/1000
            elif x == 14:
                df_s.loc[i,'trans4'] = trans_dist/1000
        else:
            lat0 = r['lat3']
            lon0 = r['lon3']
            lat1 = r['lat4']
            lon1 = r['lon4']
            body = {"coordinates":[[lon0,lat0],[lon1,lat1]],"radiuses":[-1]}
        
            call = req.post('https://api.openrouteservice.org/v2/directions/driving-car/geojson', json=body, headers=headers)
            json_obj = call.json()

            # distances in meters, get kilometers
            df_s.loc[i,'trans5'] = json_obj['features'][0]['properties']['segments'][0]['distance']/1000
        
## sometimes will error out due to application capacity constrains...run later, try at night when less web traffic


In [ ]:
df_s

In [ ]:
# save data
df_s.to_csv("Data/naatbatt_SCENARIOS_distances.csv")

# with pd.ExcelWriter("Data/naatbatt_SCENARIOS_latlon.xlsx") as writer:
#     df_s.to_excel(writer, sheet_name="SCENARIOS_dist", index=False)

In [ ]:
# sum distances to get total route driving distance
df_s['transdist (m)'] = df_s['trans1']  + df_s['trans2'] + df_s['trans3']  + df_s['trans4'] + df_s['trans5']


# distance just for west texas, e.g. no recycling, cradel to gate
df_s['cradel2gate_transdist'] = df_s['trans1']  + df_s['trans2'] + df_s['trans3']  + df_s['trans5'] # transportation to get to west Texas in km

In [ ]:
# clean data set
df_s.reset_index(inplace=True)
df_s.rename(columns = {'index':'scenarioID'},inplace=True)
df_s

In [ ]:
df_s.to_csv("Data/scenarios_final.csv") # after saving, duplicate hell's kitchen in excel and duplication with LiOH production (hell's kitchen will produce both Li2CO3 and LiOH)

### PRC route distance (port of LA to Fort Stockton)
Sea transport distance already found and inputed directly into OpenLCA process data

In [ ]:
######## PRC transport 


# port of LA to fort stockton
key = '5b3ce3597851110001cf624895482b2e40564e70a1f7588205d525d0'

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': key,
    'Content-Type': 'application/json; charset=utf-8'
}


lat0 = '33.7291858'
lon0 = '-118.2620'
fort_stockton_lat = '30.8940'
fort_stockton_lon = '-102.8793'
body = {"coordinates":[[lon0,lat0],[fort_stockton_lon,fort_stockton_lat]],"radiuses":[-1]}

call = req.post('https://api.openrouteservice.org/v2/directions/driving-car/geojson', json=body, headers=headers)

json_obj = call.json()

json_obj['features'][0]['properties']['segments'][0]['distance']/1000